In [ ]:
# Import libraries for twitter to SQL 
from twython import TwythonStreamer
import pyodbc
import json
import time

In [ ]:
# Function getting tweets  on SQL
cnx = pyodbc.connect('DSN=kubricksql') #check the DSN administrative tools in Control Panel
cursor = cnx.cursor()
tweets = []

class MyStreamer(TwythonStreamer):
    def on_success(self, data):

        if data['lang'] == 'en':
            tweets.append(data)
            cursor.execute("insert into dbo.twitssample (tweet, time) values (?, getdate())", json.dumps(data))
            cnx.commit()
            print 'received tweet #', len(tweets)

        if len(tweets) >= 10000:
            self.disconnect()


    def on_error(self, status_code, data):
        print status_code, data
        self.disconnect()


In [ ]:
# Testing the function with twitter authentication for tweets containing London and temperature 
stream = MyStreamer('zWcUk6qFeIShb0xeiTfopRkaF', 'Z2W1i597ZbS4gfCaz8V1V4OMR8GlBBhZ8yna8LyLYBxDWGf71E',
                    '775364620844490752-fqQjoDHMYbm0mQuvgv6HodwZ5MngFbo','ILnTpUbciNmOqW9alGr9aFozT8csY9O7NSpYFPOvbvThZ')
stream.statuses.filter(track=' London temperature')

In [ ]:
# Importing the tweets containing London and temperature in MongoDB instead
import pymongo
import tweepy

consumer_key = "zWcUk6qFeIShb0xeiTfopRkaF"
consumer_secret = "Z2W1i597ZbS4gfCaz8V1V4OMR8GlBBhZ8yna8LyLYBxDWGf71E"
access_key = "775364620844490752-fqQjoDHMYbm0mQuvgv6HodwZ5MngFbo"
access_secret = "ILnTpUbciNmOqW9alGr9aFozT8csY9O7NSpYFPOvbvThZ"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
tweets = 0

class CustomStreamListener(tweepy.StreamListener):
    tweets = 0
    def __init__(self, api):
        self.api = api
        super(tweepy.StreamListener, self).__init__()

        self.db = pymongo.MongoClient().Kubrick

    def on_data(self, tweet):
        self.db.tweets.insert(json.loads(tweet))
        CustomStreamListener.tweets += 1
        print CustomStreamListener.tweets

    def on_error(self, status_code):
        return True # Don't kill the stream

    def on_timeout(self):
        return True # Don't kill the stream


sapi = tweepy.streaming.Stream(auth, CustomStreamListener(api))
sapi.filter(track=['london temperature'])

In [ ]:
# Print the tweets as well
# for t in tweets:

    # print t


In [2]:
# Import dataframe and series libraries

from pandas import DataFrame, Series

# Get the tweets in Python, from SQL
cnx = pyodbc.connect('DSN=hi')
cursor = cnx.cursor()
getcmd = ("select tweet from [dbo].[tweetssample]")
cursor.execute(getcmd)
row = cursor.fetchall()
tweet_data = []
for r in row:
    tweet_data.append(json.loads(r[0]))
    
# Transform the tweets into a DataFrame and get information
df = DataFrame(tweet_data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3003 entries, 0 to 3002
Data columns (total 31 columns):
contributors                 0 non-null object
coordinates                  64 non-null object
created_at                   3003 non-null object
entities                     3003 non-null object
extended_entities            565 non-null object
favorite_count               3003 non-null int64
favorited                    3003 non-null bool
filter_level                 3003 non-null object
geo                          64 non-null object
id                           3003 non-null int64
id_str                       3003 non-null object
in_reply_to_screen_name      195 non-null object
in_reply_to_status_id        153 non-null float64
in_reply_to_status_id_str    153 non-null object
in_reply_to_user_id          195 non-null float64
in_reply_to_user_id_str      195 non-null object
is_quote_status              3003 non-null bool
lang                         3003 non-null object
place     

In [3]:
df.head()

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,id,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,None,None,Sat Sep 17 01:28:42 +0000 2016,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,776956009403281408,...,NaN,NaN,0,False,NaN,"<a href=""http://ifttt.com"" rel=""nofollow"">IFTT...","London: September 16, 2016 at 09:28PM - Partly...",1474075722443,False,"{u'follow_request_sent': None, u'profile_use_b..."
1,None,None,Sat Sep 17 01:33:12 +0000 2016,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,776957142779572224,...,NaN,NaN,0,False,NaN,"<a href=""http://dlvr.it"" rel=""nofollow"">dlvr.i...",[New London Weather] Your Weekend Forecast For...,1474075992661,False,"{u'follow_request_sent': None, u'profile_use_b..."
2,None,None,Sat Sep 17 01:33:15 +0000 2016,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,776957156603994112,...,NaN,NaN,0,False,NaN,"<a href=""http://dlvr.it"" rel=""nofollow"">dlvr.i...",[London Weather] Your Weekend Forecast For Lon...,1474075995957,False,"{u'follow_request_sent': None, u'profile_use_b..."
3,None,None,Sat Sep 17 01:39:18 +0000 2016,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,776958676309577729,...,NaN,NaN,0,False,NaN,"<a href=""http://www.weatheronline.co.uk/london...",#London Sep 17 02:00 Temperature 12C Wind NW 1...,1474076358283,False,"{u'follow_request_sent': None, u'profile_use_b..."
4,None,None,Sat Sep 17 01:50:22 +0000 2016,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,776961464154324992,...,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...","London, loving the weather... Just not when I'...",1474077022957,False,"{u'follow_request_sent': None, u'profile_use_b..."


In [4]:
# Split the json tweets
words = Series(x.split() for x in df.text)
pl=[]
for w in words:
    for l in w:
        pl.append(l)

In [5]:
path = 'C:\\Users\\student13\\Desktop\\FORECASTS\\15sep16 12pm\\FORECASTS.json'

In [6]:
# load the tweets dj = {}
with open(path) as j:
    d = json.load(j)
    dj = d

In [7]:
# Normalize the tweets
from pandas.io.json import json_normalize
result = json_normalize(dj['SiteRep']['DV']['Location']['Period'], ['Rep'], 'value')

In [8]:
# Transform the column W (weather type) in a more readable format

result['W'] = result['W'].replace('0', 'clear night')
result['W'] = result['W'].replace('1', 'sunny day')
result['W'] = result['W'].replace('2', 'partly cloudy (night)')
result['W'] = result['W'].replace('3', 'partly cloudy (day)')
result['W'] = result['W'].replace('5', 'partly cloudy (night)')
result['W'] = result['W'].replace('6', 'fog')
result['W'] = result['W'].replace('7', 'cloudy')
result['W'] = result['W'].replace('8', 'overcast')
result['W'] = result['W'].replace('9', 'light rain shower(night)')
result['W'] = result['W'].replace('10', 'light rain shower(day)')
result['W'] = result['W'].replace('11', 'drizzle')
result['W'] = result['W'].replace('12', 'light rain')
result['W'] = result['W'].replace('13', 'heavy rain shower(night)')
result['W'] = result['W'].replace('14', 'heavy rain shower(day)')
result['W'] = result['W'].replace('15', 'heavy rain')
result

,$,D,F,G,H,Pp,S,T,U,V,W,value
0,1080,SSW,25,11,70,34,7,25,1,MO,light rain shower(day),2016-09-15Z
1,1260,SSW,20,16,82,52,9,21,0,GO,light rain,2016-09-15Z
2,0,SW,19,13,89,49,7,19,0,GO,cloudy,2016-09-16Z
3,180,SSW,18,11,92,59,7,18,0,GO,heavy rain shower(night),2016-09-16Z
4,360,NW,17,13,87,38,9,18,1,MO,light rain shower(day),2016-09-16Z
5,540,NW,17,20,87,43,11,18,1,MO,light rain shower(day),2016-09-16Z
6,720,NW,16,25,77,38,13,18,3,GO,light rain shower(day),2016-09-16Z
7,900,NNW,16,25,65,32,13,19,1,VG,light rain shower(day),2016-09-16Z
8,1080,NW,15,27,60,8,16,18,1,VG,cloudy,2016-09-16Z
9,1260,NW,13,25,68,3,13,15,0,VG,clear night,2016-09-16Z


In [71]:
# Print words occureences without stopwords
from stop_words import get_stop_words
stop_words = get_stop_words('en')
s = Series(''.join(df['text']).lower().split())
scleon = Series(t for t in s if t.lower() not in stop_words)
scleon.value_counts()


weather                            1675
london                             1264
-                                   714
#london                             503
uk                                  404
temperature                         300
wind                                290
sep                                 271
..                                  266
humidity                            262
km/h                                260
rain                                230
weather:                            211
#weather                            192
day                                 191
weather.                            179
like                                172
kpa                                 156
hum:                                156
bar:                                156
wind:                               155
flash                               144
today                               127
causing                             126
nw                                  125


In [57]:
# Creating a word count for some keywords
weather = ['cloud', 'cloud.', 'cloudy.', 'cloudy', 'overcast', 'sunny', 'clear', 'thunder', 'storm', '#thunderstorm', 'thunderstorm', 'rain', 'rain.', 'pour', '#thunderstorms', 'flooding', 'flash-flooding', 'floods', 'torrential', 'sun', 'rainy', 'fog', '"thunderstorms,"', 'rainshower', 'clouds', 'raining', 'lightning', 'flood']
w = Series(t for t in scleon if t in weather)
w.value_counts()

flooding          44
rain              37
floods            15
cloudy             8
thunderstorm       5
clouds             5
sunny              4
raining            3
sun                2
cloud.             1
storm              1
overcast           1
clear              1
#thunderstorms     1
dtype: int64

In [43]:
# Get the tweets for the particular days: (15 sept, 16 sept, 17 sept, 18 sept, 19 sept)
cnx = pyodbc.connect('DSN=hi')
cursor = cnx.cursor()
getcmd = ("select tweet from [twitter].[15]")
cursor.execute(getcmd)
row = cursor.fetchall()
tweet_data = []
for r in row:
    tweet_data.append(json.loads(r[0]))

df15 = DataFrame(tweet_data)

In [58]:
cnx = pyodbc.connect('DSN=hi')
cursor = cnx.cursor()
getcmd = ("select tweet from [twitter].[16]")
cursor.execute(getcmd)
row = cursor.fetchall()
tweet_data = []
for r in row:
    tweet_data.append(json.loads(r[0]))

df16 = DataFrame(tweet_data)

In [59]:
cnx = pyodbc.connect('DSN=hi')
cursor = cnx.cursor()
getcmd = ("select tweet from [twitter].[17]")
cursor.execute(getcmd)
row = cursor.fetchall()
tweet_data = []
for r in row:
    tweet_data.append(json.loads(r[0]))

df17 = DataFrame(tweet_data)

In [60]:
cnx = pyodbc.connect('DSN=hi')
cursor = cnx.cursor()
getcmd = ("select tweet from [twitter].[18]")
cursor.execute(getcmd)
row = cursor.fetchall()
tweet_data = []
for r in row:
    tweet_data.append(json.loads(r[0]))

df18 = DataFrame(tweet_data)

In [61]:
cnx = pyodbc.connect('DSN=hi')
cursor = cnx.cursor()
getcmd = ("select tweet from [twitter].[19]")
cursor.execute(getcmd)
row = cursor.fetchall()
tweet_data = []
for r in row:
    tweet_data.append(json.loads(r[0]))

df19 = DataFrame(tweet_data)

In [66]:
# Occurrences of words for one day

s = Series(''.join(df19['text']).lower().split())
scleon = Series(t for t in s if t.lower() not in stop_words)
scleon.value_counts()
w = Series(t for t in scleon if t in weather)
w.value_counts()

sunny       160
cloudy       20
rain         17
fog          14
clouds       10
sun           8
rainy         6
overcast      4
clear         4
rain.         2
raining       2
storm         1
cloudy.       1
dtype: int64

In [67]:
# Creating stacked bar charts for most occuring words from the 15th till 19th september

from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[40, 256, 35, 15, 27],
    name='Rain'
)
trace2 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[14, 37, 38, 39, 31],
    name='Cloudy'
)
trace3 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[7, 169, 3, 2, 1],
    name='Thunderstorm'
)
trace4 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[59, 224, 1, 0, 0],
    name='Floods'
)
trace5 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[1, 5, 33, 38, 4],
    name='Overcast'
)
trace6 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[6, 29, 6, 7, 168],
    name='Sunny'
)
data = [trace1, trace2, trace3, trace4, trace5, trace6]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='twitter')

In [68]:
# Manually creating stacked bar charts for most occuring words from the 15th till 19th september

from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[2, 4, 0, 0, 0],
    name='Rain'
)
trace2 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[0, 2, 6, 8, 8],
    name='Cloudy'
)
trace3 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[0, 1, 0, 0, 0],
    name='Thunderstorm'
)
trace4 = go.Bar(
    x=['2016-09-15', '2016-09-16', '2016-09-17', '2016-09-18', '2016-09-19'],
    y=[0, 1, 2, 0, 0],
    name='Clear Night'
)

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='twitterforecasts')

In [69]:
type(scleon)

pandas.core.series.Series

In [96]:
# Occurence of selected words
scleon.value_counts()
weather = ['cloud', 'cloud.', 'cloudy.', 'cloudy', 'overcast', 'sunny', 'clear', 'thunder', 'storm', '#thunderstorm', 'thunderstorm', 'rain', 'rain.', 'pour', '#thunderstorms', 'flooding', 'flash-flooding', 'floods', 'torrential', 'sun', 'rainy', 'fog', '"thunderstorms,"', 'rainshower', 'clouds', 'raining', 'lightning', 'flood']
w = Series(t for t in scleon if t in weather)
w.value_counts()

rain              230
cloudy            124
#thunderstorms    118
flooding          113
flash-flooding     85
overcast           80
floods             69
torrential         55
sunny              37
rainy              33
storm              31
sun                30
fog                28
rainshower         25
clouds             20
raining            18
clear              18
lightning          17
thunder            17
flood              14
thunderstorm       12
rain.              10
cloudy.            10
#thunderstorm       3
dtype: int64

In [103]:
# Doing some manual cleaning w.replace(to_replace='flash-flooding',inplace = True, value='flooding')
w.replace(to_replace='#thunderstorms',inplace = True, value='thunderstorms')
w.replace(to_replace='#thunderstorm',inplace = True, value='thunderstorms')
w.replace(to_replace='rainy',inplace = True, value='rain')
w.replace(to_replace='storm',inplace = True, value='thunderstorms')
w.replace(to_replace='sun',inplace = True, value='sunny')
w.replace(to_replace='rainshower',inplace = True, value='rain')
w.replace(to_replace='clouds',inplace = True, value='cloudy')
w.replace(to_replace='raining',inplace = True, value='rain')
w.replace(to_replace='lightning',inplace = True, value='thunderstorms')
w.replace(to_replace='thunder',inplace = True, value='thunderstorms')
w.replace(to_replace='flood',inplace = True, value='flooding')
w.replace(to_replace='floods',inplace = True, value='flooding')
w.replace(to_replace='cloudy.',inplace = True, value='cloudy')
w.replace(to_replace='rain.',inplace = True, value='rain')
w.replace(to_replace='thunderstorm',inplace = True, value='thunderstorms')
l = w.value_counts().to_dict().keys()
v = w.value_counts().to_dict().values()

In [104]:
# Most occurring words in tweets
labels = l
values = v
trace = go.Pie(labels=labels, values=values)
py.iplot([trace])